<a href="https://colab.research.google.com/github/VenissaCarolQuadros/COMP551/blob/dev1/COMP551_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
def read_data(url, dest):
  response = requests.get(url)
  with open(dest, 'wb') as file:
      file.write(response.content)
  return

In [3]:
dest1= 'LinearRegression.xlsx'
read_data("https://raw.githubusercontent.com/VenissaCarolQuadros/COMP551/main/data/ENB2012_data.xlsx", dest1)
df1= pd.read_excel(dest1)

dest2= 'LogisticRegression.csv'
read_data('https://raw.githubusercontent.com/VenissaCarolQuadros/COMP551/main/data/Qualitative_Bankruptcy/Qualitative_Bankruptcy.data.txt', dest2)
df2= pd.read_csv(dest2, header=None)


In [4]:
print(df1.shape, df2.shape) ##Verifying dataset size
print( df1[df1.eq('?').any(1)],  df2[ df2.eq('?').any(1)]) #Checking for nulls
##No nulls found. Dataset size seems right

(768, 10) (250, 7)
Empty DataFrame
Columns: [X1, X2, X3, X4, X5, X6, X7, X8, Y1, Y2]
Index: [] Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []


##Linear Regression

In [96]:
class GradientDescent:
    
    def __init__(self, learning_rate=.0001, max_iters=1e4, epsilon=1e-8, record_history=False):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.record_history = record_history
        self.epsilon = epsilon
        if record_history:
            self.w_history = []                 #to store the weight history for visualization
            
    def run(self, gradient_fn, x, y, w):
        grad = np.inf
        t = 1
        
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            print(np.linalg.norm(grad))
            grad = gradient_fn(x, y, w)               # compute the gradient with present weight
            w = w - self.learning_rate * grad         # weight update step
            if self.record_history:
                self.w_history.append(w)
            t += 1
        return w

class LinearRegression:
  def __init__(self):
    ()
  
  def editData(self):
    self.X= np.c_[np.ones((self.X).shape[0]), (self.X)] 
    return
  
  def fitClosedForm(self, X, Y):
    self.X=X
    self.Y=Y
    self.editData()
    self.w= np.linalg.inv((self.X).T @ (self.X))@ (self.X).T@ (self.Y)
    return

  """
  def fitGD(self, X, Y, iterations=1000, alpha=0.01):
    self.fitClosedForm(X, Y) #Is this a good way tp find an initial w?
    gradJ= (1/len(self.Y))*np.sum(((self.X @ self.w)- Y)*self.X)
    self.w= 

    return

  """

  def fitGD(self, X, Y, optimizer1, optimizer2):
        self.fitClosedForm(X, Y)
        def gradient(x, y, w):                          # define the gradient function
            yh =  x @ w 
            N, D = x.shape
            grad = .5*np.dot(yh - y, x)/N
            return grad
        w0 = np.zeros(self.X.shape[1])                            # initialize the weights 
        self.w[:,0] = optimizer1.run(gradient, self.X, self.Y[:,0], w0)      # run the optimizer to get the optimal weights
        self.w[:,1] = optimizer2.run(gradient, self.X, self.Y[:,1], w0) 
        return

  def predict(self, Xt):
    Xt= np.c_[np.ones((Xt).shape[0]), (Xt)] 
    return Xt @ self.w
  
  

In [75]:
#Train test split
train=df1.sample(frac=0.8,random_state=200)
test=df1.drop(train.index)

model1= LinearRegression()

X_train= train.iloc[:, :8].to_numpy(copy=True) 
Y_train= train.iloc[:, 8:].to_numpy(copy=True)

model1.fitClosedForm(X_train, Y_train)

X_test= test.iloc[:, :8].to_numpy(copy=True)
Y_test= test.iloc[:, 8:].to_numpy(copy=True)

Ypredict=model1.predict(X_test)
#print(np.c_[Y_test, Ypredict])
loss=np.sum((Y_test-Ypredict)**2, axis=0)
print(loss)

[306629.53199871 378009.33937901]


In [108]:
optimizer1 = GradientDescent(learning_rate=.00001, max_iters=100, record_history=True)
optimizer2 = GradientDescent(learning_rate=.00001, max_iters=100, record_history=True)
model= LinearRegression()
model.fitGD(X_train, Y_train, optimizer1, optimizer2)

inf
8406.8138023492
16279.709307489291
31536.295765228348
61096.03964402311
118365.54816387239
229319.0377322104
444278.7716332435
860738.4083358252
1667580.6474034342
3230743.8127793665
6259190.923821794
12126455.50313207
23493599.24049835
45516120.122076005
88182196.76857124
170842765.2913631
330988017.10572344
641250844.194639
1242349039.6301694
2406906985.3755655
4663102760.536846
9034220053.972492
17502752174.863613
33909549674.959206
65695814445.10662
127278010972.61467
246586364960.6015
477732445061.2747
925551131347.7902
1793147829072.6091
3474015673478.317
6730501916183.237
13039565822796.209
25262644445313.004
48943439761972.695
94822230543575.64
183707059597492.56
355911093342690.56
689536409988450.9
1335896715761994.8
2588144743819365.0
5014229869663981.0
9714488050126318.0
1.8820692415198576e+16
3.646290583299522e+16
7.064264546995141e+16
1.3686192158819822e+17
2.6515407876084736e+17
5.1370523420720614e+17
9.952442326557454e+17
1.92817011912126e+18
3.735605679774979e+18
7.

In [101]:
Ypredict=model.predict(X_test)
#print(np.c_[Y_test, Ypredict])
loss=np.sum((Y_test-Ypredict)**2, axis=0)
print(loss)

[3.66480409e+292 4.46032424e+292]
